In [1]:
# add modules to Python's search path
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)


import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KernelDensity  # display as density curves

import  torch
from geomloss import SamplesLoss
import tensorflow as tf
from modules import wasserstein as tfw

[pyKeOps]: Warning, no cuda detected. Switching to cpu only.


In [2]:
use_cuda = torch.cuda.is_available()
# N.B.: We use float64 numbers to get nice limits when blur -> +infinity
dtype    = torch.cuda.DoubleTensor if use_cuda else torch.DoubleTensor

# make a convenient wrapper for producing samples in form of a tensor
def torch_sampler(mean, cov, size):
    samples = np.random.multivariate_normal(mean, cov, size)
    return torch.from_numpy(samples)

# set up parameters for our two test distributions
dimension = 3
mean_1 = np.zeros(dimension)
mean_2 = mean_1 + 0.1 * np.ones(dimension)
cov_1 = np.identity(dimension)
cov_2 = cov_1

# finally create the samplers our test distributions
sampler_1 = lambda size: torch_sampler(mean_1, cov_1, size)
sampler_2 = lambda size: torch_sampler(mean_2, cov_2, size)

# test our samplers
print("samples from distribution #1:\n{}".format(sampler_1(3)))
print("samples from distribution #2:\n{}".format(sampler_2(3)))

samples from distribution #1:
tensor([[ 1.1202, -0.7755,  1.6073],
        [ 0.1669, -1.5277,  2.3073],
        [ 1.1081,  0.5839,  0.7476]], dtype=torch.float64)
samples from distribution #2:
tensor([[ 1.0109, -0.7399,  0.3451],
        [ 1.7252, -0.4314, -0.3702],
        [-0.8072, -0.9464, -1.2067]], dtype=torch.float64)


In [3]:
num_samples_1 = 500
num_samples_2 = 500
samples_1 = sampler_1(num_samples_1)
samples_2 = sampler_2(num_samples_2)
loss = SamplesLoss("sinkhorn", p=2, blur=0.01, scaling=.99, backend="online")
print(np.sqrt(loss(samples_1, samples_2).item()))

0.3680344296264513


In [4]:
# make a convenient wrapper for producing samples in form of a tensor
def tf_sampler(mean, cov, size):
    samples = np.random.multivariate_normal(mean, cov, size)
    return tf.convert_to_tensor(samples, dtype=tf.float32)

# set up parameters for our two test distributions
dimension = 3
mean_1 = np.zeros(dimension)
mean_2 = mean_1 + 0.0 * np.ones(dimension)
cov_1 = np.identity(dimension)
cov_2 = cov_1

# finally create the samplers our test distributions
sampler_1 = lambda size: tf_sampler(mean_1, cov_1, size)
sampler_2 = lambda size: tf_sampler(mean_2, cov_2, size)

# test our samplers
print("samples from distribution #1:\n{}".format(sampler_1(3)))
print("samples from distribution #2:\n{}".format(sampler_2(3)))

samples from distribution #1:
[[-1.2604666   2.025482   -2.666641  ]
 [-0.8957441   0.5420875   1.5477074 ]
 [ 0.829818   -0.05294777  0.25876302]]
samples from distribution #2:
[[-1.1012024   1.8941648   1.5154725 ]
 [ 0.47532982  0.8626874  -0.32130912]
 [ 0.08357774 -0.28164318  1.8414799 ]]


In [9]:
num_samples_1 = 50
num_samples_2 = 200
samples_1 = sampler_1(num_samples_1)
samples_1 -= tf.reduce_mean(samples_1, axis=0)
samples_2 = sampler_2(num_samples_2)
samples_2 -= tf.reduce_mean(samples_2, axis=0)
print(tf.sqrt(tf.reduce_sum((tf.reduce_mean(samples_1, axis=0) - tf.reduce_mean(samples_2, axis=0))**2)))
loss = tfw.sinkhorn_loss(samples_1, samples_2, epsilon=0.01, niter=2000, p=2)
print(np.sqrt(loss))

tf.Tensor(1.5359042e-08, shape=(), dtype=float32)
0.7816565


In [6]:
samples_1 -= tf.reduce_mean(samples_1, axis=0)

In [7]:
tf.reduce_mean(samples_1, axis=0)

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([ 3.8146974e-09, -1.0490417e-08,  0.0000000e+00], dtype=float32)>